# GPU_IVF_FLAT
- GPU_IVF_FLAT 인덱스는 GPU 환경 전용으로 설계된 IVF_FLAT 인덱스의 GPU 가속 버전
- 벡터 데이터를 nlist 클러스터 단위로 분할하고 먼저 대상 쿼리 벡터를 각 클러스터의 중심과 비교하여 유사성을 계산
- nprobe 매개변수를 조정하면 가장 가능성이 높은 클러스터만 검색되므로 정확도와 속도 간의 균형을 유지하면서 쿼리 시간 단축

참고자료: https://milvus.io/docs/ko/gpu-ivf-flat.md

## 인덱스 구축
- Milvus의 벡터 필드에 GPU_IVF_FLAT 인덱스를 구축하려면 add_index() 방법을 사용하여 index_type, metric_type 및 인덱스에 대한 추가 매개 변수를 지정

In [7]:
from pymilvus import MilvusClient, DataType

index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="vector", # Name of the vector field to be indexed
    index_type="GPU_IVF_FLAT", # Type of the index to create
    index_name="vector_index", # Name of the index to create
    metric_type="L2", # Metric type used to measure similarity
    params={
        "nlist": 1024, # Number of clusters for the index
    } # Index building params
)

In [8]:
search_params = {
    "params": {
        "nprobe": 10, # Number of clusters to search
    }
}

client = MilvusClient(
    uri="http://localhost:19530"
)

if client.has_collection("my_collection"):
    client.drop_collection("my_collection")

schema = client.create_schema(
    auto_id=False,
    enable_dynamic_schema=False,
)

schema.add_field(field_name="index", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=5)
schema.add_field(field_name="scalar_1", datatype=DataType.INT64)
schema.add_field(field_name="scalar_2", datatype=DataType.INT32)

client.create_collection(
    collection_name="my_collection",
    schema=schema,
    index_params=index_params
)

client.load_collection("my_collection")

res = client.search(
    collection_name="my_collection", # Collection name
    anns_field="vector",
    data=[[0.1, 0.2, 0.3, 0.4, 0.5]],  # Query vector
    limit=3,  # TopK results to return
    search_params=search_params
)


2025-07-08 04:18:39,460 [ERROR][handler]: RPC error: [create_index], <MilvusException: (code=1100, message=invalid parameter[expected=valid index][actual=invalid index type: GPU_IVF_FLAT])>, <Time:{'RPC start': '2025-07-08 04:18:39.458587', 'RPC error': '2025-07-08 04:18:39.460762'}> (decorators.py:140)
2025-07-08 04:18:39,461 [ERROR][_create_index]: Failed to create an index on collection: my_collection (milvus_client.py:183)


MilvusException: <MilvusException: (code=1100, message=invalid parameter[expected=valid index][actual=invalid index type: GPU_IVF_FLAT])>